In [1]:
from pyspark import SparkContext
sc = SparkContext()

22/05/08 23:04:15 WARN Utils: Your hostname, simba resolves to a loopback address: 127.0.0.1; using 147.96.81.245 instead (on interface ext)
22/05/08 23:04:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/08 23:04:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
def get_edges(line): # grafo no dirigido, sin bucles.
    edge = line.strip().split(',')
    n1 = edge[0]
    n2 = edge[1]
    if n1 < n2:
         return (n1,n2)
    elif n1 > n2:
         return (n2,n1)
    else:
        pass #n1 == n2

In [10]:
def get_rdd_distict_edges(sc, filename):
    return sc.textFile(filename).\
        map(get_edges).\
        filter(lambda x: x is not None).\
        distinct()

In [11]:
get_rdd_distict_edges(sc, "g0.txt")

PythonRDD[35] at RDD at PythonRDD.scala:53

In [12]:
def adjacents(sc, filename):
    nodes = get_rdd_distict_edges(sc, filename)
    adj = nodes.groupByKey().mapValues(list)
    return adj

In [13]:
adjacents(sc, "g0.txt")

PythonRDD[46] at RDD at PythonRDD.scala:53

In [14]:
""" Apartado 1 """

def get_info_adjacents(tupla):
    lista = []
    for i in range(len(tupla[1])):
        lista.append(((tupla[0],tupla[1][i]), 'existe'))
        if len(tupla[1]) > 1:
            for m in range(len(tupla[1])):
                for k in range(m+1,len(tupla[1])):
                    if tupla[1][m] >= tupla[1][k]:
                        lista.append(((tupla[1][k], tupla[1][m]),('pendiente', tupla[0])))
                    else:
                        lista.append(((tupla[1][m], tupla[1][k]),('pendiente', tupla[0])))
    return lista

def get_info_triciclos(tupla):
    triciclo = []
    for i in tupla[1]:
        if i != 'existe':
            triciclo.append([tupla[0][0],tupla[0][1],i[1]])
    return triciclo

def get_triciclos(rdd):
    return rdd.flatMap(get_info_adjacents).\
        distinct().\
        groupByKey().\
        sortByKey().\
        mapValues(list).\
        filter(lambda x: len(x[1])>1 and 'existe' in x[1]).\
        flatMap(get_info_triciclos)

def triciclos(sc, filename):
    adj = adjacents(sc, filename)
    triciclos = get_triciclos(adj)
    return triciclos.collect()


In [15]:
triciclos(sc, "g4.txt")

[['B', 'C', 'A'], ['C', 'E', 'A'], ['D', 'E', 'C'], ['E', 'F', 'A']]

In [16]:
""" Apartado 2 """

def main(sc, files):
    rdd_lista = []
    for filename in files:
        file_rdd = get_rdd_distict_edges(sc,filename).\
            distinct()
        rdd_lista.append(file_rdd)
    adj = sc.union(rdd_lista).\
        distinct().\
        groupByKey().\
        mapValues(list).\
        sortByKey().\
        map(lambda x: (x[0], sorted(x[1])))
    solucion = get_triciclos(adj)
    return solucion.collect()

In [17]:
main(sc, ["g0.txt","g1.txt"])

[['B', 'C', 'A'], ['B', 'D', 'A'], ['C', 'D', 'A'], ['C', 'D', 'B']]

In [18]:
""" Apartado 3 """

def get_rdd_distict_edges_aux(sc, filename):
    return sc.textFile(filename).\
        map(get_edges).\
        filter(lambda x: x is not None).\
        distinct().\
        map(lambda x: (x[0]+filename,x[1]+filename))
    
def varios_ficheros(sc,files):
    rdd_lista = []
    for filename in files:
        file_rdd = get_rdd_distict_edges_aux(sc, filename).\
            distinct()
        rdd_lista.append(file_rdd)
    adj = sc.union(rdd_lista).\
        distinct().\
        groupByKey().\
        mapValues(list).\
        sortByKey().\
        map(lambda x: (x[0], sorted(x[1])))
    solucion = get_triciclos(adj)
    return solucion.collect()

In [25]:
varios_ficheros(sc, ["g0.txt","g1.txt"])

[['Bg0.txt', 'Cg0.txt', 'Ag0.txt'],
 ['Bg1.txt', 'Cg1.txt', 'Ag1.txt'],
 ['Bg1.txt', 'Dg1.txt', 'Ag1.txt'],
 ['Cg1.txt', 'Dg1.txt', 'Ag1.txt'],
 ['Cg1.txt', 'Dg1.txt', 'Bg1.txt']]